## Ejercicio 4: Modelado Predictivo 

### By:
Auberth Eduardo Hurtado

### Date:
2025-01-17

### Description:

Create a predictive model

## 📚 Required Libraries

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

# Load Data
Load the cleaned_sales_data.csv file from the datamart/data/final directory and inspect the first few rows of the dataset.

In [ ]:
# Load the cleaned sales data
sales_data = pd.read_csv("../data/final/cleaned_sales_data.csv")
sales_data

## 💾 1. Realiza un análisis exploratorio avanzado: 
- Identifica estacionalidad y tendencias. 
- Detecta outliers y evalúa su impacto en las ventas.


In [ ]:
# Identifying seasonality and trends
sales_data['date'] = pd.to_datetime(sales_data['order_date'])
data = sales_data.set_index('date')

monthly_revenue = data['total_revenue'].resample('ME').sum()
monthly_revenue.plot(figsize=(14, 7), title='Monthly Revenue')
plt.show()

In [ ]:
# Analysis by month
monthly_decomposition = seasonal_decompose(monthly_revenue, model='additive')
# Plot the decomposed components
plt.figure(figsize=(14, 10))

plt.subplot(411)
plt.plot(monthly_decomposition.observed, label='Observed')
plt.legend(loc='upper left')

plt.subplot(412)
plt.plot(monthly_decomposition.trend, label='Trend')
plt.legend(loc='upper left')

plt.subplot(413)
plt.plot(monthly_decomposition.seasonal, label='Seasonal')
plt.legend(loc='upper left')

plt.subplot(414)
plt.plot(monthly_decomposition.resid, label='Residual')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()


This result suggests the existence of a stochastic function. In other words, total revenue can be estimated as $Revenue = trend + seasonality - 3.5$.

In [ ]:
# Analysis by day
daily_revenue = data['total_revenue'].resample('D').sum()
daily_decomposition = seasonal_decompose(daily_revenue, model='additive', period=365)

# Plot the decomposed components
plt.figure(figsize=(14, 10))

plt.subplot(411)
plt.plot(daily_decomposition.observed, label='Observed')
plt.legend(loc='upper left')

plt.subplot(412)
plt.plot(daily_decomposition.trend, label='Trend')
plt.legend(loc='upper left')

plt.subplot(413)
plt.plot(daily_decomposition.seasonal, label='Seasonal')
plt.legend(loc='upper left')

plt.subplot(414)
plt.plot(daily_decomposition.resid, label='Residual')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

This result suggests the existence of no erros. In other words, daily total revenue can be estimated as $trend + seasonality$.

In [ ]:
# Detection of outliers and evaluation of their impact on sales
plt.figure(figsize=(14, 7))
sns.boxplot(data=data, x='region', y='total_revenue')
plt.title('Boxplot of Total Revenue by Region')
plt.show()

Outliers have a positive impact by increasing the total revenue in the final estimation. If we exclude the outliers, the average revenue will decrease.

## 2. Define una estrategia para predecir el ingreso diario total: 
- Divide los datos en entrenamiento (80%) y prueba (20%). 
- Implementa modelos: 
    - Regresión lineal múltiple considerando quantity, price, discount, y region.  
    - Modelo basado en Random Forest o Gradient Boosting para capturar relaciones no lineales. 
- Opcional: Usa técnicas de feature engineering, como generación de variables temporales.

In [42]:
# Feature Engineering
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['day_of_month'] = data.index.day
data['week_of_year'] = data.index.isocalendar().week

In [43]:
# Convert categorical variables to dummy variables
data = pd.get_dummies(data, columns=['region', 'product_id'], drop_first=True)
# for testing purposes other option is use one hot encoder of scikit learn

In [44]:
# Splitting data in train and test sets
X = data[['quantity', 'price', 'discount', 'day_of_week', 'month', 'day_of_month', 'week_of_year'] + [col for col in data.columns if 'region_' in col or 'product_id_' in col]]
y = data['total_revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [45]:
#X_train.head()
#X_test.head()
#y_train.head()
#y_test.head()


In [46]:
# Models implementation

# A. Linear regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

# B. Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# C. Gradient Boosting
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

For the purposes of the test, I have run the models with their default parameters; however, some adjustments could be made to improve the initial comparison.

# 3. Evalúa los modelos usando: 
- MAE, R^2, y análisis de residuales. 
- Describe el rendimiento y si el modelo es aplicable en un entorno real. 

In [47]:
# Auxiliar function to evaluate the models

def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'{model_name} - MAE: {mae:.2f}, R^2: {r2:.2f}')
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_true, y=y_pred, lowess=True, line_kws={'color': 'red', 'lw': 1})
    plt.title(f'Residuals Plot for {model_name}')
    plt.xlabel('Actual Values')
    plt.ylabel('Residuals')
    plt.show()

In [ ]:
# Evaluar Regresión Lineal Múltiple
evaluate_model(y_test, y_pred_linear, 'Linear Regression')

# Evaluar Random Forest
evaluate_model(y_test, y_pred_rf, 'Random Forest')

# Evaluar Gradient Boosting
evaluate_model(y_test, y_pred_gb, 'Gradient Boosting')


# 💡 Descripción del rendimiento y aplicabilidad en un entorno real
Descripción del rendimiento y aplicabilidad en un entorno real:
- **La regresión lineal múltiple** puede ser útil para relaciones lineales simples, pero puede no capturar todas las complejidades, en este caso se observa que el modelo falla en estimar los ingresos con valores más pequeños presentando un problema de heterocedasticidad.
- **Random Forest** y **Gradient Boosting** son más efectivos para capturar relaciones no lineales y pueden ofrecer mejores predicciones, entre los dos le fue mejor a Random forest al encontrar bajor error en sus estimaciones.
- Evaluar el MAE y $R^2$ ayuda a entender la precisión y la capacidad explicativa de los modelos, aunque considero que $R^2$ debería ser revisado como medida de bondad de ajuste al final de todo el proceso pues es muy sensible a los supuestos del modelo, idealmente acompañarlo con otras medidas como RMSE o MAPE.
- En un entorno real, se debe considerar la interpretabilidad del modelo, el tiempo de entrenamiento y la capacidad de generalización. El modelo elegible podría ser random forest pero faltaría profundizar más y examinar otras alternativas como el ajuste de hiperparámetros u otros modelos más idones para este tipo de problemas.

----

- **Multiple Linear Regression** can be useful for simple linear relationships but may not capture all complexities. In this case, the model struggles to estimate revenues with smaller values, presenting a problem of heteroscedasticity.

- **Random Forest** and **Gradient Boosting** are more effective in capturing non-linear relationships and can provide better predictions. Among the two, Random Forest performed better, exhibiting lower error in its estimations.

- Evaluating MAE and $R^2$ helps in understanding the accuracy and explanatory power of the models. However $R^2$ should be reviewed as a goodness-of-fit measure at the end of the entire process, as it is very sensitive to the model's assumptions. Ideally, it should be complemented with other measures such as RMSE or MAPE.

- In a real-world environment, model interpretability, training time, and generalization capability should be considered. While Random Forest could be a viable model, further exploration and examination of other alternatives, such as hyperparameter tuning or other more suitable models for this type of problem, are necessary.

## 💡 Be cautious

In [49]:
# Save the final and processed dataset
data.to_csv(r'../data/final/final_sales_data.csv', index=False)

In [ ]:
# Save the final model
joblib.dump(rf_model, r'../models/random_forest_model.pkl')